In [49]:
def splitPeptide(peptide):
    length = len(peptide)
    splits = []
    
    for i in range(0, length):
        character = peptide[i]
        toAdd=""
        toAdd+=character
        splits.append(toAdd)
        for j in range(i+1, length):
            toAdd+=peptide[j]
            if(splitSize(toAdd, 11)):
                splits.append(toAdd)
    return splits
            
            
    

In [50]:
def combinePeptide(splits):
    combine = []
    for i in range(0, len(splits)):
        toAdd=""
        for j in range(i+1, len(splits)):
            toAdd+=splits[i]
            toAdd+=splits[j]
            if(splitSize(toAdd, 12)):
                
                combine.append(toAdd)
            
            toAdd = ""
            toAdd+=splits[j]
            toAdd+=splits[i]
            if(splitSize(toAdd, 12)):
                
                combine.append(toAdd)
        
            toAdd=""
    return combine
     
    

In [51]:
def splitSize(split, size):
    if(len(split)>size):
        return False
    return True

In [81]:
#splits = splitPeptide("AKLjf111111jhjhjhjhjhjhjhjh1112hjhfjfjhfjhkkkkkkkkkkrjhjhjhytytytujjjjjju")
six  = ""
for i in range(0, 600):
    six+="1"
splits = splitPeptide(six)

In [82]:

print(len(splits))

6545


In [83]:
combine = combinePeptide(splits)

print(len(combine))

23492910
